# pre-preprocessing 2-photon data with texture experiments

In [ ]:
# processing 2p imaging data for texture representation in collaboration with Dr.Gardner@Standord
# modified from pre-preprocessingVSExperiment.ipynb
# last update @190626
# written by RA

import scipy.io as sio
%matplotlib nbagg
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm_notebook as tqdm
import cv2
import seaborn as sns
sns.set()
sns.set_style("ticks")
from pims import ND2_Reader
import tifffile


# Change the following line to the name of the consecutive (same ROI) experiments

In [ ]:
# experiment = "M190213_18222"
# iexp = '1'
# iseries = ('4','5','6','8','9','10')
# experiment = "M190421_18228"
# iseries = '2'
# expset = ('4','5','6','8','9','10')
#iseries = '1'
#expset = ('3','4','5','7','8','9')
experiment = "M190625_18273"
iseries = '1'
expset = ('2')
expType = ('texture',)
#expType = ('ori','spont','driver','driver','spont','ori')
for i in range(len(expset)):
    foo = experiment+'_'+iseries+'_'+expset[i]
    print(foo, ' is ', expType[i])

# Read the filenames for data log, 2 photon data, and the ET video

In [ ]:
master_folder_data_log = "//Labserver/data/MOUSE/LOGS/VS_LOGS" # for Win, not work in Linux
master_folder_nd2 = "//NCB-LABSERVER6/data/MOUSE/IMAGING/GCAMP/"
#master_folder_nd2 = "//LABSERVER5/data/MOUSE/IMAGING/GCAMP/"
master_folder_ET = "//NCB-LABSERVER6/data/MOUSE/EyeTracking/GCaMP/"
#print(os.path.join(master_folder_data_log, experiment))
#len(os.listdir(os.path.join(master_folder_data_log, experiment)))
      

In [ ]:
# experimental setting
do_ET = False
is_driverCell = False

data_log_filename = []
nd2_filename = []
ET_filename =[]
ntri = [] # #trial
for i in range(len(expset)): # loop for each iseries
    data_log_filename.append(os.path.join(master_folder_data_log, experiment+'_'+iseries+'_'+expset[i]+'.mat'))
    assert os.path.isfile(data_log_filename[-1]), 'No log data found!!: '+data_log_filename[-1]
    print('found a data_log:',os.path.join(master_folder_data_log, experiment+'_'+iseries+'_'+expset[i]+'.mat'))
            
    nd2_filename.append(os.path.join(master_folder_nd2, experiment, experiment+'-'+iseries+'-'+expset[i]+'.nd2'))
    assert os.path.isfile(nd2_filename[-1]), 'No nd2 data found!!: '+nd2_filename[-1]
    print('found a nd2 file:', nd2_filename[-1])
    
    if do_ET:
        
        print(os.path.join(master_folder_ET, experiment,iseries,expset[i]))
        ET_filename_tmp = []
        ET_subdir = os.path.join(master_folder_ET, experiment,iseries,expset[i])
        n_ETfiles = len([name for name in os.listdir(ET_subdir) if os.path.isfile(os.path.join(ET_subdir,name)) and name.endswith('.eye')])
        for j in range(n_ETfiles):
            ET_filename_tmp.append(os.path.join(ET_subdir, experiment+'_'+str(j+1)+'.eye'))
            assert os.path.isfile(ET_filename_tmp[-1]), 'No EyeTracking data found!!'
        
        assert (len(ET_filename_tmp) == 195 and expType[i] == 'ori') or (len(ET_filename_tmp) == 1 and expType[i] == 'spont') or (len(ET_filename_tmp) == 1 and expType[i] == 'driver') or (len(ET_filename_tmp) == 1 and expType[i] == 'driver_30ms'), 'Mismatch trial number and number of .eye files'
        ET_filename.append(ET_filename_tmp)
        ntri.append(len(ET_filename_tmp))
    else: # HACK for texture exp...
        ntri.append(96)


# Create the folders where all the post-processed data will be saved

In [ ]:
master_analysis_folder = "//NCB-LABSERVER6/data/MOUSE/IMAGING/GCAMP/" # same folder as raw .nd2 data
experiment_folder = os.path.join(master_analysis_folder, experiment)
folders_to_make = ["ET", "2P", "Preprocessed"] # put everything in this folder


for folder in  folders_to_make:
    new_path = os.path.join(master_analysis_folder, experiment_folder, folder)
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    

# Convert Eye video to .mp4

In [ ]:
#plt.figure()
if do_ET:
    ET_n_frame = []
    for j in range(len(expset)):
        ET_n_frame_tmp = []
        for i in tqdm(range(len(ET_filename[j]))):
            try: 
                new_ET_foldername = os.path.join(experiment_folder, "ET/" ,iseries)
                new_ET_filename = os.path.join(new_ET_foldername, experiment+'_'+expset[j]+'_'+ str(i+1)+".mp4")
                ET_video = open_raw_eye_file(ET_filename[j][i])
                if not os.path.exists(new_ET_foldername):
                    os.makedirs(new_ET_foldername)
                ET_n_frame_tmp.append(ET_video.n_frames)
                if not os.path.exists(new_ET_filename):
                    out = cv2.VideoWriter(new_ET_filename,\
                                  cv2.VideoWriter_fourcc('H','2','6','4'), 30, (ET_video.width, ET_video.height))

                    for frame in ET_video.img_data:
                        out.write(cv2.cvtColor(frame,cv2.COLOR_GRAY2RGB))
                    out.release()
            except:
                print(ET_filename[j][i], ' can not be converted properly...')
                ET_n_frame_tmp.append(0) # assign no frame for errornous trial.
        ET_n_frame.append(ET_n_frame_tmp)


In [ ]:
# optional: concatenate all sessions for training dataset
# ET_n_frame = []
# new_ET_filename = os.path.join(experiment_folder, "ET/", experiment+".mp4")
# out = cv2.VideoWriter(new_ET_filename,\
#                       cv2.VideoWriter_fourcc('H','2','6','4'), 30, (ET_video.width, ET_video.height))
# for i in range(len(ET_filename)):
#     ET_video = open_raw_eye_file(ET_filename[i])
#     print(i," th trial's number of frames: ", ET_video.n_frames)
#     ET_n_frame.append(ET_video.n_frames)
#     #    plt.imshow(ET_video.img_data[100])
#     for frame in ET_video.img_data:
#         out.write(cv2.cvtColor(frame,cv2.COLOR_GRAY2RGB))
# out.release()

# Convert nd2 to tiff for suite2p

In [ ]:
twoP_video = []; n_frame = []
for i in range(len(expset)):
    twoP_video.append(ND2_Reader(nd2_filename[i]))
    n_frame.append(twoP_video[-1].sizes['t'])
print(np.sum(n_frame), ' frames in total')

In [ ]:
# check 2p image
# plt.figure()
# plt.title(twoP_video[0].sizes)
# plt.imshow(twoP_video[0][1000], cmap='gray', vmin=0, vmax=1500)

Create empty tensor to put the entire nd2 file in, then we will save that tensor as a tiff file.

Shape of tensor is (time, y, x)

In [ ]:
all_2p_frames = np.zeros((np.sum(n_frame), twoP_video[0].sizes['y'], twoP_video[0].sizes['x']), dtype=np.uint16)
for j in range(len(expset)):
    for i in tqdm(range(n_frame[j])):   
        all_2p_frames[int(np.sum(n_frame[:j]))+i] = twoP_video[j][i]


Let's save the tensor

In [ ]:
print(os.path.join(experiment_folder, "2P/", experiment+'_'+iseries, experiment+'_'+iseries+".tiff"))
assert not os.path.isfile(os.path.join(experiment_folder, "2P/", experiment+'_'+iseries, experiment+'_'+iseries+".tiff")), 'Tiff file already exsit!'
os.makedirs(os.path.join(experiment_folder, "2P/", experiment+'_'+iseries))
tifffile.imsave(data=all_2p_frames, file=os.path.join(experiment_folder, "2P/", experiment+'_'+iseries, experiment+'_'+iseries+".tiff"), bigsize=True)

RUN suite2p

In [ ]:
# run suite2p!

# Read Data Log and obtain alignment vectors to stimulus onset

In [ ]:
data_logs = []
for i in range(len(expset)):
    data_logs.append(sio.loadmat(data_log_filename[i]))
dac_sampling_rate = 10000 #10Khz sampling rate of 2p DAQ

For information on what each column of logging contains go here:
http://172.17.150.6/mediawiki/index.php?title=2P

The first column is the timestamps of the DAQ

In [ ]:
if do_ET:
    ET_timestamp = []
    for iexp in range(len(expset)):
        # detect the timestamps for each frame, removing the missing frames (on the assumption that last one is missing...)
        frameEx_ET_ttl = data_logs[iexp]['AIdata'][:, 8] # rise for 5 msec at frame onset
        ET_on_off_trial_ttl = data_logs[iexp]['AIdata'][:, 7]
        # TTL signal goes from zero to a value of around 3.
        # So we find the point
        ET_frame_ttl_inTrial = frameEx_ET_ttl.copy()
        ET_frame_ttl_inTrial[ET_on_off_trial_ttl < 1.0 ] = 0.0 # pad non-trial period with 0 
        diff_frameEx_ttl = np.hstack((0, np.diff(ET_frame_ttl_inTrial)))

        # sanity check that missing frame is no more than 1 in each trials
    #    frame_onset_idx = np.where(diff_frameEx_ttl < -1.5)[0] # fall in ttl
        frame_onset_idx = np.where(diff_frameEx_ttl > 1.5)[0] # rise in ttl (in trial)
        frame_onset_idx_whole = np.where(np.hstack((0, np.diff(frameEx_ET_ttl))) > 1.5)[0] # rise in ttl (in and out of trial)
        gap_frame_idx = np.hstack((0,np.where(np.diff(frame_onset_idx)>0.5*dac_sampling_rate)[0]+1,frame_onset_idx.shape[0]-1))
        assert np.max(np.diff(gap_frame_idx)-np.array(ET_n_frame[iexp])) < 2, str(np.max(np.diff(gap_frame_idx)-np.array(ET_n_frame[iexp])))+' missing frames in some trials!!'


        # extract ET frame onset timestamps
        ET_timestamp_sub = []
        for itr in range(len(ET_n_frame[iexp])):
            strt_idx = np.where(frame_onset_idx_whole >= frame_onset_idx[gap_frame_idx[itr]])[0][0]
            ET_timestamp_sub.append(data_logs[iexp]['AIdata'][frame_onset_idx_whole[strt_idx:(strt_idx+ET_n_frame[iexp][itr])],0])
    #        ET_timestamp_sub.append(data_logs[iexp]['AIdata'][frame_onset_idx[gap_frame_idx[itr]]:frame_onset_idx[gap_frame_idx[itr]]+ET_n_frame[iexp][itr],0])
        ET_timestamp.append(ET_timestamp_sub)


Loading all the trial information and converting it to uncorrected indeces of the DAC:

# Now extract 2p frame onset timing (complicated)

In [ ]:
twoP_frame_onset =[]
for j in range(len(expset)):
    signal_2p_aq = data_logs[j]['AIdata'][:, 2].copy()

    # Values of 2p ttl fluctuate from around 3 when not aqcuiring to 0 while frame aqcuiring.
    signal_2p_aq[np.where(signal_2p_aq<1)] = 0
    signal_2p_aq[np.where(signal_2p_aq>1)] = 1
    diff_2p_ttl = np.diff(signal_2p_aq)


    twoP_ttl_rise = np.where(diff_2p_ttl > 0.5)[0]
    twoP_ttl_fall = np.where(diff_2p_ttl < -0.5)[0]
    twoP_frame_onset_tmp = []
    for i in tqdm(range(twoP_ttl_rise.shape[0])): # rise and fall within 20 timebin (=2msec) is defined as frame 
        if np.any(np.all(np.vstack((twoP_ttl_fall>twoP_ttl_rise[i],twoP_ttl_fall<twoP_ttl_rise[i]+20)),axis=0)):
            twoP_frame_onset_tmp.append(data_logs[j]['AIdata'][twoP_ttl_rise[i],0])
    twoP_frame_onset.append(twoP_frame_onset_tmp)
    
    # find the number of frames for each repeats 
    # start/end of each trial was defined as frame interval > 100 msec
    int_thresh = 0.1 # 100 msec for interval thresholding 
    frame_int = np.diff(twoP_frame_onset[j])
    twoP_frame_repeat = np.hstack((0, np.cumsum(frame_int>int_thresh)))
 
    #Now let's assert that the detected frame number from ttl matchs frame number of nd2 data.
    print('detected TTL peaks : ',len(twoP_frame_onset[j]))
    print('actual frame number: ',n_frame[j])
    # the mismatch in number is ok, as 2p acqusition is stopped manually and can continues after VS end
    #assert len(twoP_frame_onset[j])==n_frame[j] or expType[j]!='ori', 'Missing frames....'
   # print(len(twoP_frame_onset[j]),n_frame[j] )

 First assertion, do the number of trials logged by BVS matches the number of times the photodiode black square shows up

In [ ]:
#assert ACTUAL_stim_onset_indeces.shape[0] == int_stim_onset_block_trial.shape[0]

Second assertion, is the difference in stimulus onset of BVS and actual stimulus onset as measured by the photodiode within one second?

In [ ]:
#assert (np.abs(ACTUAL_stim_onset_indeces-int_stim_onset_block_trial) < dac_sampling_rate).all()

# Open  suite2p analyzed 2p Data and splice it by trial type

In [ ]:
# is this section necessary? (can separately load npy files for analysis...)
suite2p_F_filename = os.path.join(experiment_folder, "2P", experiment+'_'+iseries, "suite2p/plane0/F.npy")
suite2p_is_cell_filename = os.path.join(experiment_folder, "2P", experiment+'_'+iseries, "suite2p/plane0/iscell.npy")
suite2p_spks_filename = os.path.join(experiment_folder, "2P", experiment+'_'+iseries, "suite2p/plane0/spks.npy")
suite2p_stat_filename = os.path.join(experiment_folder, "2P", experiment+'_'+iseries,  "suite2p/plane0/stat.npy")
try:
    raw_traces = np.load(suite2p_F_filename)
    spike_traces = np.load(suite2p_spks_filename)
    is_cell = np.load(suite2p_is_cell_filename)
    stat = np.load(suite2p_stat_filename)
except FileNotFoundError:
    print("ERROR: Please ensure that you've ran suite2p on the 2p tiff file before running this cell")

In [ ]:
# extract roi info from stat.npy
nroi = stat.shape[0]
twoP_rois = np.zeros((512,512,nroi)) # note that after motion correction data size can be smaller than 512 x 512
twoP_med = np.zeros((2,nroi)) 
for iroi in range(nroi):
    twoP_med[:,iroi] = stat[iroi]['med']
    for ipix in range(stat[iroi]['xpix'].shape[0]):
        twoP_rois[stat[iroi]['ypix'][ipix], stat[iroi]['xpix'][ipix],iroi] = 1

In [ ]:
if is_driverCell:
    dc_idx = 24 # index for driver_cell (0-indexing)

# deeplabcut things

In [ ]:
# do it on GPU server

# save everything in hdf5 format

In [ ]:
# save everything in HDF5 to load in matlab
hdf5_filename = os.path.join(experiment_folder,'Preprocessed/'+experiment+'_'+iseries+'.h5')
import h5py
with h5py.File(hdf5_filename, 'w') as f:
    dt= h5py.special_dtype(vlen=float) # I don't know what it means...
#     f.create_dataset('raw_traces', data = raw_traces)
#     f.create_dataset('spike_traces', data = spike_traces)
#     f.create_dataset('is_cell', data = is_cell)
 #   f.create_dataset('stat', data = stat)
    f.create_dataset('twoP_med', data = twoP_med)
    f.create_dataset('twoP_rois', data = twoP_rois)
    f.create_dataset('twoP_n_frame',data=n_frame)
    expset_list = [int(x) for x in expset]
    f.create_dataset('expSet',data=np.array(expset_list))
#    f.create_dataset('expType',data=expType)
    if is_driverCell:
        f.create_dataset('dc_idx',data=dc_idx+1) # change to 1-indexing.
    f.create_dataset('ntri',data=ntri) 
    for i in range(len(expset)):
        grp = f.create_group(str(expset[i]))
        grp.create_dataset('twoP_frame_onset',data = np.array(twoP_frame_onset[i]))
        grp.create_dataset('expType', data = expType[i])
        if do_ET:
            if len(ET_timestamp[i])==1:
                grp.create_dataset('ET_timestamp', data = ET_timestamp[i])
            else:
                grp.create_dataset('ET_timestamp', data = ET_timestamp[i],dtype=dt) # dtype for list of >1 elements
            if i > 0: print('a'); continue # hack in calse len(expSet)==1
    #         grp.create_dataset('pupilCenterX', data =pupilCenterX[i],dtype=dt)
    #         grp.create_dataset('pupilCenterY', data =pupilCenterY[i],dtype=dt)
    #         grp.create_dataset('pupilArea', data = pupilArea[i],dtype=dt)
            grp.create_dataset('ET_n_frame',data=ET_n_frame[i])


In [ ]:
# END 

In [ ]:
# memo


# Temporary Function Definitions

In [ ]:
class TwoPhotonTrial:
    def __init__(self, experiment, feedback_type, orientation, start_time, stim_time, response_time, end_time,
                 feedback_ended_time, list_of_cell_traces=None, list_of_ET_frames=None):
        self.experimet = experiment
        self.feedback_type = feedback_type
        self.orientation = orientation
        self.start_time = start_time
        self.stim_time = stim_time
        self.response_time = response_time
        self.end_time = end_time
        self.list_of_cell_traces = list_of_cell_traces
        self.list_of_ET_frames = list_of_ET_frames
        self.length = end_time - start_time
        
    def __gt__(self, obj):
        return self.length > obj.length

    def __ge__(self, obj):
        return self.length >= obj.length

    def __lt__(self, obj):
        return self.length < obj.length

    def __le__(self, obj):
        return self.length <= obj.length

    def get_feedback_type_string(self):
        if self.feedback_type == -1:
            return "INCORRECT"
        elif self.feedback_type == 0:
            return "TIMEOUT"
        elif self.feedback_type == 1:
            return "CORRECT"
        else:
            raise IndexError("Unkown feedback type:", self.feed_back_type)

In [ ]:
class Video:
    def __init__(self, height, width, n_frames, img_data, time_stamps):
        self.height = height
        self.width = width
        self.n_frames = n_frames
    
        self.img_data = img_data
        self.time_stamps = time_stamps

def open_raw_eye_file(filename):
    with open(filename, 'rb') as fid:
        version = int(np.frombuffer(fid.read(4), dtype=np.uint32)[0])
        width = int(np.frombuffer(fid.read(4), dtype=np.uint32)[0])
        height = int(np.frombuffer(fid.read(4), dtype=np.uint32)[0])
        n_frames = int(np.frombuffer(fid.read(4), dtype=np.uint32)[0])
        pixel_format = int(np.frombuffer(fid.read(4), dtype=np.uint32)[0])
        
        if pixel_format == 8:
            img_data = np.frombuffer(fid.read(height*width*n_frames), dtype=np.uint8)
            img_data = np.reshape(img_data, (n_frames, height, width))
        elif pixel_format == 16:
            # Convert it to 8bits so we can easily save it to mp4.
            img_data = np.frombuffer(fid.read(height*width*n_frames*2), dtype=np.uint16)/256
            img_data = img_data.astype(np.uint8)
            img_data = np.reshape(img_data, (n_frames, height, width))
        else:
            print(pixel_format)
            raise NotImplemented("This bit format is not yet implemented, fix it Fed! >:()")
        
        time_stamps = np.frombuffer(fid.read(n_frames*4), dtype=np.uint32)


    return Video(height, width, n_frames, img_data, time_stamps)


def calculate_df_f0(traces, baseline=None, std_norm=False):
    # Use entire video for baseline if no baseline given
    if baseline is None:
        baseline = np.mean(traces, axis=1)

    # calculate dF
    df = traces.T-baseline
    
    
    if not std_norm:
        df = df/baseline
    else:
        df = df/(1+(np.std(traces, axis=0))/np.sqrt(traces.shape[0]))
        #df = df/(1+(np.std(frames, axis=0)))
            
            
    return df.T

In [ ]:
# temp memo
ET_video = open_raw_eye_file(ET_filename[0][1])
ET_video.img_data.shape
filename = ET_filename[0][0]
print(filename)


In [ ]:
a=open(filename,'rb')
version = int(np.frombuffer(a.read(4), dtype=np.uint32)[0])
width = int(np.frombuffer(a.read(4), dtype=np.uint32)[0])
height = int(np.frombuffer(a.read(4), dtype=np.uint32)[0])
n_frames = int(np.frombuffer(a.read(4), dtype=np.uint32)[0])
pixel_format = int(np.frombuffer(a.read(4), dtype=np.uint32)[0])
print(version)
print(width)
print(height)
print(n_frames)
print(pixel_format)